# みんかぶ経済指標

In [2]:
import pandas as pd
import datetime
# from newspaper import Article
import time
import re
import datetime
import numpy as np

In [3]:
def get_df(url, date):
    url_date = url + date
    df = pd.read_html(url_date)
    df = df[0].dropna(subset = [4]).drop(2,axis =1)
    df.columns = ["発表時刻", "経済指標", "前回変動幅(USD/JPY)","前回","予想","結果"]
    df["日付"] = date
    return df

In [4]:
def str2num(text):
    '''
    表中の単位付きデータから数値を抜き出す
    データのカラムにapplyする
    かっこの中身を優先して読み取る。（速報値用）
    args:
        text: string or int or float
    return:
        num: float
    '''
    if((type(text) == int) or (type(text) == float)):
        return text
    if len(text.split('（')) == 2:
        text = text.split('（')[1]
    pattern = '[+]?[-]?[0-9]+[.]?[0-9]+?'
    result = re.match(pattern, text)
    if result == None:
        return 0
    return  float(text[result.start():result.end()])

In [5]:
def inv_del_dup(df):
    '''
    データを古い順番に並べ替えて、重複データを削除する。
    新しい順だと新しいデータが残ってしまう。
    args:
        df: pd.DataFrame, columns = ['発表時刻', '経済指標', '前回変動幅(USD/JPY)', '前回', '予想', '結果', '日付']
    return:
        df: reversed, del duplicates
    '''
    df = df.reset_index(drop=True)
    cols = df.columns
    idx = list(df.index.values)
    idx.reverse()
    
    def inv_bool(a):
        if a == False:
            return True
        else:
            return False
    
    df_rev = df.iloc[idx].reset_index(drop=True)
    return df_rev[df_rev[cols[:-1]].duplicated().apply(inv_bool)].reset_index(drop=True)

In [6]:
def datetime_merge(d, t):
    dt = d + " " + t
    return datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M')

In [7]:
def indicators(text):
    '''
    経済指標を成型して、指標を取り出す。
    '''
    if len(text.split('[')) > 1:
        return "-"
    return text.split(' ')[0]
    
def ind_filter(df, ind):
    '''
    指定した指標、国名のデータを取り出す。
    '''
    return df[df['経済指標'].str.contains(ind, na=False)]

In [11]:
import lxml.html

In [12]:
N = 820
pdf = pd.DataFrame()
for i in range(N):
    date = datetime.date(2021, 4, 1) - datetime.timedelta(days=i)
    date = date.strftime('%Y-%m-%d')
    url = f'https://fx.minkabu.jp/indicators?date='
    pdf_date = get_df(url, date)
    pdf = pd.concat([pdf, pdf_date], axis=0)
    time.sleep(0.1)
pdf = pdf.reset_index(drop=True)

ImportError: lxml not found, please install it

In [8]:
pdf

,発表時刻,経済指標,前回変動幅(USD/JPY),前回,予想,結果,日付
0,00:00,中国・貿易収支 01月,+0.9pips,3293億元,---,---,2020-03-07
1,00:00,カナダ・Ivey購買担当者景況感指数 02月,+1.8pips,57.3,---,54.1,2020-03-07
2,00:00,アメリカ・卸売在庫（確報値） 01月,+1.8pips,-0.2%,-0.2%,-0.4%,2020-03-07
3,12:05,中国・貿易収支 02月,-5.1pips,---,2290億元,-426億元,2020-03-07
0,00:00,アメリカ・製造業新規受注 01月,-1.0pips,1.8% （1.9%）,-0.1%,-0.5%,2020-03-06
1,00:00,アメリカ・耐久財受注（確報値） 01月 [前月比],-1.0pips,-0.2%,-0.2%,-0.2%,2020-03-06
2,00:00,[コア・前月比],NaN,0.9%,0.8%,0.8%,2020-03-06
3,06:30,豪・AiGサービス業指数 02月,+0.1pips,47.4,---,47.0,2020-03-06
4,08:30,日本・全世帯家計調査 01月,-1.1pips,-4.8%,-4.0%,-3.9%,2020-03-06
5,09:30,豪・小売売上高 01月,+0.3pips,-0.5% （-0.7%）,0.0%,-0.3%,2020-03-06


In [ ]:
file_name = 'fx_indicators.csv' 
pdf.to_csv(file_name, index=False)
# pdf = pd.read_csv(file_name)
# del pdf['Unnamed: 0']

In [ ]:
pdf_rev = inv_del_dup(pdf)
display(pdf_rev)

In [ ]:
ind_list = pdf['経済指標'].apply(indicators).unique()
ind_list = ind_list[(ind_list != '-')]

In [ ]:
ind = ind_list[10]
pdf_filtered = ind_filter(pdf_rev, ind)

ind_prev_arr = pdf_filtered['前回'].apply(str2num).values
ind_result_arr = pdf_filtered['結果'].apply(str2num).values

time_arr =  pdf_filtered.apply(lambda row:datetime_merge(*row[['日付', '発表時刻']]), axis=1).values
ind_pred_arr = pdf_filtered['予想'].apply(str2num).values
ind_arr = np.hstack([ind_prev_arr[1:], (ind_result_arr[-1])])

In [ ]:
pdf_filtered